In [ ]:
import os
from dotenv import load_dotenv

from pprint import pprint

from src.config import LLMConf, EmbedderConf, KnowledgeGraphConfig
from src.factory.embeddings import get_embeddings
from src.graph.knowledge_graph import KnowledgeGraph
from src.agents.graph_qa import GraphAgentResponder
env = load_dotenv('config.env')

from pgs.utils import get_configuration_from_env

from src.utils.logger import get_logger
logger = get_logger(__name__)

conf = get_configuration_from_env()

In [ ]:
kg_config = KnowledgeGraphConfig(
    uri=os.getenv("NEO4J_URI"),
    user=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
    index_name='vector'
)

embedder_conf = EmbedderConf(
    type=os.getenv("EMBEDDINGS_TYPE"),
    model=os.getenv("EMBEDDINGS_MODEL_NAME"),
)

model_conf=LLMConf(
    type=os.getenv("RE_MODEL_TYPE"),
    model=os.getenv("RE_MODEL_NAME"), 
    temperature=os.getenv("RE_MODEL_TEMPERATURE"), 
    deployment=os.getenv("RE_MODEL_DEPLOYMENT"),
    api_key=os.getenv("RE_API_KEY"),
    endpoint=os.getenv("RE_MODEL_ENDPOINT"),
    api_version=os.getenv("RE_MODEL_API_VERSION") or None
)

embeddings = get_embeddings(conf=embedder_conf)

kg = KnowledgeGraph(conf=kg_config,embeddings_model=embeddings)

## Cypher Chains
Here we use a method to answer using  only Cypher queries

In [ ]:
responder = GraphAgentResponder(
    qa_llm_conf=conf.qa_model,
    cypher_llm_conf=conf.qa_model,
    graph=kg,
    # rephrase_llm_conf=conf.qa_model
)

In [ ]:
responder.answer_with_cypher(query="Who works for Europe Direct?")

In [ ]:
responder.answer_with_cypher(
    query="What legislation has been discussed that regards Apple?", 
    intermediate_steps=True
)

In [ ]:
# rephraser not working as expected
responder_with_rephrase = GraphAgentResponder(
    qa_llm_conf=conf.qa_model,
    cypher_llm_conf=conf.qa_model,
    graph=kg,
    rephrase_llm_conf=conf.qa_model
)

In [ ]:
responder_with_rephrase.answer_with_cypher(query="Who works for Europe Direct?")

In [ ]:
responder_with_rephrase.answer_with_cypher(
    query="What legislation has been discussed that regards Apple?", 
    intermediate_steps=True
)

## Vanilla RAG
Uses only vanilla RAG to answer the user's question.  
If `use_adjacent_chunks=True` will query the graph for additional context   
compared to the Chunks retrieved by the similarity search. Latency will be higher due to expanded context. 

In [ ]:
responder = GraphAgentResponder(
    qa_llm_conf=conf.qa_model,
    cypher_llm_conf=conf.qa_model,
    graph=kg
    # rephrase_llm_conf=conf.qa_model
)

In [ ]:
responder.answer_with_context("Who works for Europe Direct?")

In [ ]:
responder.answer_with_context(
    query="Who works for Europe Direct?", 
    use_adjacent_chunks=True
)

In [ ]:
responder.answer_with_context(
    query="What legislation has been discussed that regards Apple?"
)

In [ ]:
responder.answer_with_context(
    query="What legislation has been discussed that regards Apple?",
    use_adjacent_chunks=True
)

In [ ]:
responder.answer_with_context("Who is Eva Hrncirova?")

In [ ]:
responder.answer_with_context("Who is Eva Hrncirova?", use_adjacent_chunks=True)

In [ ]:
responder.answer_with_context("Who is the Commissioner for Intergenerational Fairness?")

In [ ]:
responder.answer_with_context(
    query="Who is the Commissioner for Intergenerational Fairness?",
    use_adjacent_chunks=True
)

In [ ]:
responder.answer_with_context(query="Who is Glenn Micallef?")

In [ ]:
responder.answer_with_context(query="Who is Glenn Micallef?", use_adjacent_chunks=True)

In [ ]:
responder.answer_with_context("what is the Digital Markets Act?")

In [ ]:
responder.answer_with_context("what is the Digital Markets Act?", use_adjacent_chunks=True)

## Community Reports  

Queries two vector indexes to get the user's answer out of an ensemble of contexts:
1. one made of a list of `CommunityReport`
2. one made of a list of `Chunk` from the same communities of the reports. 

If `use_adjacent_chunks=True` will query the graph for additional context 
compared to the Chunks retrieved by the similarity search. Latency will be higher due to expanded context. 

In [ ]:
responder = GraphAgentResponder(
    qa_llm_conf=conf.qa_model,
    cypher_llm_conf=conf.qa_model,
    graph=kg
    # rephrase_llm_conf=conf.qa_model
)

In [ ]:
responder.answer_with_community_reports(
    query="what is the Digital Markets Act?", 
    use_adjacent_chunks=False, 
    community_type="leiden"
)

In [ ]:
responder.answer_with_community_reports(
    query="what is the Digital Markets Act?", 
    use_adjacent_chunks=True,
    community_type="leiden"
)

In [ ]:
responder.answer_with_community_reports(query="What is the European Commission telling Apple to do?")

In [ ]:
responder.answer_with_community_reports(
    query="What is the European Commission telling Apple to do?", 
    use_adjacent_chunks=True,
    community_type="leiden"
)

In [ ]:
responder.answer_with_community_reports(
    query="Who is the Commissioner for Intergenerational Fairness?", 
    use_adjacent_chunks=True,
    community_type="leiden"
)

## Answer with Communities Subgraph

Answers after querying for communities:
* read the most relevant community report 
* fetch chunks belonging to the most relevant community (the one from the community report)
* follow the MENTIONS relationship of each Chunk and obtain a dictionary 
* fetch the community subgraph under the form of another dictionary 
* passes the dictionaries + the report to a reconciler agent to decide how to answer

In [ ]:
responder = GraphAgentResponder(
    qa_llm_conf=conf.qa_model,
    cypher_llm_conf=conf.qa_model,
    graph=kg
    # rephrase_llm_conf=conf.qa_model
)

In [ ]:
responder.answer_with_community_subgraph(query="What is the European Commission telling Apple to do?")

In [ ]:
responder.answer_with_community_subgraph(query="Who is the Commissioner for Intergenerational Fairness?")

## Full Answer

In [ ]:
responder = GraphAgentResponder(
    qa_llm_conf=conf.qa_model,
    cypher_llm_conf=conf.qa_model,
    graph=kg
    # rephrase_llm_conf=conf.qa_model
)

In [ ]:
responder.answer(query="What is Europe doing for remote areas?")

In [ ]:
responder.answer(query="Who is the Commissioner for Intergenerational Fairness?")

In [ ]:
responder.answer(query="Who is the Commissioner for Intergenerational Fairness?", use_adjacent_chunks=True)

In [ ]:
responder.answer(query="Who works for Europe Direct?", use_adjacent_chunks=True)

In [ ]:
responder.answer(query="Who is Eva Hrncirova?", use_adjacent_chunks=True)

In [ ]:
responder.answer(query="What is the European Commission telling Apple to do?")